In [1]:
import pandas as pd
import talib as ta
import MetaTrader5 as mt5

In [21]:
if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()


In [186]:
ativo = 'EURUSD'
df1 = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M30, 0,999))
df1['data'] = pd.to_datetime(df1['time'], unit='s')
df = df1[df1['data'] > '2022-01-01']
df = df[['data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2022-12-05 21:30:00,1.04875,1.04940,1.04812,1.04920,0
1,2022-12-05 22:00:00,1.04920,1.04969,1.04811,1.04813,0
2,2022-12-05 22:30:00,1.04814,1.04910,1.04797,1.04846,0
3,2022-12-05 23:00:00,1.04847,1.04912,1.04836,1.04905,0
4,2022-12-05 23:30:00,1.04904,1.04939,1.04867,1.04867,0
...,...,...,...,...,...,...
994,2023-01-05 14:30:00,1.06116,1.06170,1.05978,1.05990,0
995,2023-01-05 15:00:00,1.05991,1.06076,1.05776,1.05795,0
996,2023-01-05 15:30:00,1.05797,1.05797,1.05509,1.05519,0
997,2023-01-05 16:00:00,1.05519,1.05616,1.05467,1.05496,0


In [190]:
timeframe = 5
ativo = f'EURUSD{timeframe}'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(rf'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks\Dados\{ativo}.csv', names = colnames, sep='\t')
df = df[df['Data'] > '2022-06-01']

#https://www.alpharithms.com/calculate-macd-python-272222/

In [191]:
#macd é a lenta
#macdsignal é a rápida

pontos = 0.0005
rate_stop = 1
rate_tp = 2

df['macd'], df['macdsignal'], df['macdhist'] = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA_200'] = ta.SMA(df['Close'],timeperiod=30)
df['SAR'] = ta.SAR(df['High'],df['Low'], acceleration=0, maximum=0)

'''#df['macd_diff'] = (df['macd'] - df['macdsignal'])
df['macd_position'] = df.apply(lambda x: 1 if x['macd'] > x['macdsignal'] else -1 if x['macd'] < x['macdsignal'] else 0, axis = 1)
df['macd_diff'] = df['macd_position'].diff()
# 2 - cruzamento pra cima || -2 cruzamento pra baixo

df['cruzamento_diff_position'] = df['macdsignal'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
#'cruzamento_diff_position' --> 1 se linhas acima do 0 '''

df['cruzamento_hist'] = df['macdhist'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
df['cruzamento_hist'] = df['cruzamento_hist'].diff()

df['cruzamento_dist'] = abs(df['macdsignal'])

'''df['acao'] = df.apply(lambda x: 'call' if (x['cruzamento_diff_position'] == -1 and x['macd_diff'] == 2 and x['Close'] > x['EMA_200']) else
                                'sell' if (x['cruzamento_diff_position'] == 1 and x['macd_diff'] == -2 and x['Close'] < x['EMA_200']) else 0,axis = 1)
'''

df['acao'] = df.apply(lambda x: 'call' if (x['cruzamento_hist'] == 2 and x['Close'] < x['SAR'] ) else
                                'sell' if (x['cruzamento_hist'] == -2 and x['Close'] > x['SAR']) else 0,axis = 1)

df['stop'] = df.apply(lambda x: x['Close'] + (pontos * rate_stop)  if x['acao'] == 'sell' else 
    x['Close'] - (pontos * rate_stop) if x['acao'] == 'call' else 0, axis = 1)


df['tp'] = df.apply(lambda x: x['Close'] - rate_tp * pontos if x['acao'] == 'sell' else
                                            x['Close'] + rate_tp * pontos if x['acao'] == 'call' else 0, axis =1)


for index, row in df.iterrows():
        if row['acao'] == 'call':
            i = 1
            while True:
                try:
                    if df.loc[index + i,'Low'] <= row['stop']:
                        df.loc[index,'resultado_binario'] = 0 
                        break
                    elif df.loc[index + i,'High'] >= row['tp']:
                        df.loc[index,'resultado_binario'] = 1
                        break
                    i+=1
                except:
                    df.loc[index,'resultado_binario'] = 0
                    break
        elif row['acao'] == 'sell':
            i = 1
            while True:
                try:
                    if df.loc[index + i,'High'] >= row['stop']:
                        df.loc[index,'resultado_binario'] = 0
                        break
                    elif   df.loc[index + i,'Low'] <= row['tp']:
                        df.loc[index,'resultado_binario'] = 1
                        break
                    i+=1
                except:
                    df.loc[index,'resultado_binario'] = 0
                    break
df_acao = df[(df['acao'] != 0)]

In [192]:
df_acao['resultado_binario'].sum()/df_acao.shape[0]

0.3311160384331116

In [177]:
df_acao[['Data','acao','Close','stop','tp','resultado_binario']].tail()

,Data,acao,Close,stop,tp,resultado_binario
881,2023-01-05 06:40:00,call,1.06080,1.06070,1.06100,0.0
891,2023-01-05 07:30:00,call,1.06039,1.06029,1.06059,1.0
922,2023-01-05 10:05:00,call,1.06070,1.06060,1.06090,0.0
939,2023-01-05 11:30:00,call,1.06235,1.06225,1.06255,1.0
972,2023-01-05 14:15:00,call,1.06149,1.06139,1.06169,0.0


In [99]:
df_acao.columns

Index(['Data', 'Open', 'High', 'Low', 'Close', 'Vol', 'macd', 'macdsignal',
       'macdhist', 'macd_position', 'macd_diff', 'cruzamento_diff_position',
       'cruzamento_dist', 'acao', 'stop', 'tp', 'resultado_binario'],
      dtype='object')

In [53]:
import plotly.express as px

In [112]:
fig = px.histogram(df_acao, x="cruzamento_dist", color="resultado_binario")
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [106]:
df_new = df_acao[df_acao['cruzamento_dist'] < 0.0001]
df_new['resultado_binario'].sum()/df_new.shape[0]

0.1022307376488299

In [ ]:
df_acao[['Data','acao','Close','stop','tp','macdhist','resultado_binario']].tail(10)

In [167]:
df_acao[['Close','SAR']]

,Close,SAR
166258,1.07110,1.07393
166259,1.07128,1.07393
166260,1.07048,1.07393
166261,1.07097,1.07393
166262,1.07081,1.07393
...,...,...
199995,1.03558,1.07738
199996,1.03551,1.07738
199997,1.03549,1.07738
199998,1.03532,1.07738
